[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-filing-exhibits-python.ipynb)

# Access SEC Filing Exhibits and Attachments with Python -- Free, No API Key

Use **edgartools** to access exhibits and attachments from SEC filings -- completely free, no API key or paid subscription required. Every SEC filing contains multiple documents: the main filing, exhibits like material agreements, subsidiaries lists, certifications, and press releases.

**What you'll learn:**
- Browse all documents and exhibits in any SEC filing
- Download exhibit text, HTML, or raw content
- Access the primary filing document
- Read press releases from 8-K earnings filings
- Extract subsidiary lists (EX-21) and other key exhibits
- Compare exhibit counts across companies

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Browse All Exhibits in a Filing

Every SEC filing comes with multiple documents. The `attachments` property gives you access to all of them -- exhibits, graphics, XBRL data, and the primary filing document:

In [ ]:
filing = Company("AAPL").get_filings(form="10-K")[0]

print(f"Filing:      {filing.form} filed {filing.filing_date}")
print(f"Attachments: {len(filing.attachments)} total")
print(f"Exhibits:    {len(filing.attachments.exhibits)}")
print(f"Graphics:    {len(filing.attachments.graphics)}")

filing.attachments

## List Key Exhibits

Exhibits are the most useful attachments -- they contain material agreements (EX-10), subsidiaries (EX-21), auditor consent (EX-23), and officer certifications (EX-31, EX-32):

In [ ]:
for exhibit in filing.attachments.exhibits:
    if not exhibit.document_type.startswith("EX-101"):  # Skip XBRL taxonomy files
        print(f"{exhibit.document_type:10s}  {exhibit.document}")

## Read Exhibit Content

Download any exhibit and extract its text. Here we read the subsidiaries list (EX-21), which shows all entities owned by the company:

In [ ]:
for exhibit in filing.attachments.exhibits:
    if exhibit.document_type == "EX-21.1":
        print(f"Exhibit:  {exhibit.document_type}")
        print(f"Document: {exhibit.document}")
        print(f"URL:      {exhibit.url}")
        print(f"HTML:     {exhibit.is_html()}\n")

        text = exhibit.text()
        print(f"Content ({len(text):,} chars):\n")
        print(text[:1000])
        break

## Access the Primary Filing Document

The primary document is the main filing itself (the 10-K, 10-Q, or 8-K). Access it directly, or use the convenience methods for full text, markdown, or HTML:

In [ ]:
primary = filing.attachments.primary_html_document
print(f"Primary document: {primary.document} ({primary.document_type})")

# Convenience methods on the filing itself
text = filing.text()
markdown = filing.markdown()
html = filing.html()

print(f"\nPlain text: {len(text):>10,} chars")
print(f"Markdown:   {len(markdown):>10,} chars")
print(f"HTML:       {len(html):>10,} chars")

## Read Press Releases from 8-K Filings

When companies report earnings, they file an 8-K with the press release as an EX-99.1 exhibit. Extract the full text:

In [ ]:
eightk = Company("MSFT").get_filings(form="8-K")[0]

print(f"8-K filed: {eightk.filing_date}")
print(f"Exhibits:")
for e in eightk.attachments.exhibits:
    print(f"  {e.document_type:10s} {e.document}")

# Read the press release (EX-99.1)
for e in eightk.attachments.exhibits:
    if e.document_type.startswith("EX-99"):
        text = e.text()
        print(f"\nPress release ({len(text):,} chars):\n")
        print(text[:600])
        break

## Filter Attachments by Type

Use the `query()` method to filter attachments by document type, or access pre-filtered collections for common categories:

In [ ]:
filing = Company("AAPL").get_filings(form="10-K")[0]

# Query for specific exhibit types
ex_docs = filing.attachments.query('document_type.startswith("EX-") and not document_type.startswith("EX-101")')
print(f"Human-readable exhibits: {len(ex_docs)}")
for e in ex_docs:
    print(f"  {e.document_type:10s} {e.document}")

# Pre-filtered collections
print(f"\nPrimary documents: {len(filing.attachments.primary_documents)}")
print(f"Data files (XBRL):  {len(filing.attachments.data_files)}")
print(f"Graphics (images):  {len(filing.attachments.graphics)}")

## Compare Exhibits Across Companies

Different companies file different exhibits depending on their contracts, management changes, and disclosure requirements:

In [ ]:
tickers = ["AAPL", "MSFT", "NVDA", "GOOG"]
rows = []

for ticker in tickers:
    filing = Company(ticker).get_filings(form="10-K")[0]
    exhibits = filing.attachments.exhibits
    ex_types = [e.document_type for e in exhibits
                if e.document_type.startswith("EX-") and not e.document_type.startswith("EX-101")]
    rows.append({
        "Ticker": ticker,
        "Filed": str(filing.filing_date),
        "Total Attachments": len(filing.attachments),
        "Key Exhibits": len(ex_types),
        "Exhibit Types": ", ".join(ex_types),
    })

pd.DataFrame(rows).set_index("Ticker")

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing SEC filing exhibits:

**With edgartools (free, no API key):**
```python
filing = Company("AAPL").get_filings(form="10-K")[0]
filing.attachments.exhibits        # All exhibits
exhibit.text()                      # Extract text from any exhibit
exhibit.url                         # Direct SEC download URL
filing.text()                       # Full filing as plain text
```

**Typical manual approach:**
```python
import requests
from bs4 import BeautifulSoup
# ... find filing index page on EDGAR,
# ... parse HTML table to find exhibit links,
# ... download each exhibit separately, extract text manually
```

With edgartools, all filing attachments are indexed and accessible -- browse, filter, and read any exhibit with clean Python methods.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Access attachments ──
filing = Company("AAPL").get_filings(form="10-K")[0]
filing.attachments                     # All attachments
filing.attachments.exhibits            # Exhibits only
filing.attachments.primary_documents   # Primary filing documents
filing.attachments.data_files          # XBRL and data files
filing.attachments.graphics            # Images

# ── Individual attachment ──
exhibit = filing.attachments.exhibits[0]
exhibit.document_type                  # "EX-21.1", "EX-99.1", etc.
exhibit.document                       # Filename
exhibit.description                    # Exhibit description
exhibit.url                            # SEC download URL
exhibit.is_html()                      # True for HTML documents

# ── Read content ──
exhibit.text()                         # Extracted plain text
exhibit.download()                     # Raw HTML/content
exhibit.markdown()                     # Converted to markdown

# ── Primary document shortcuts ──
filing.text()                          # Full filing as text
filing.markdown()                      # Full filing as markdown
filing.html()                          # Full filing as HTML

# ── Filter attachments ──
filing.attachments.query('document_type.startswith("EX-99")')  # Press releases
```

## What's Next

You've learned how to access exhibits and attachments from SEC filings. Here are related tutorials:

- [Extract 8-K Earnings Releases](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/8k-earnings-release-python.ipynb)
- [SEC Filing Text Extraction for NLP](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-filing-text-nlp-python.ipynb)
- [Download SEC Filings in Bulk](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-sec-filings-bulk-python.ipynb)
- [Analyze 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/analyze-10k-annual-report-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*